In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load gun data
gun_data = pd.read_csv('gun-violence-data-cleaned.csv')
gun_data = gun_data.drop(['Unnamed: 0'], axis=1)

gun_data.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,congressional_district,gun_stolen,gun_type,...,longitude,n_guns_involved,participant_age,participant_gender,participant_name,participant_relationship,participant_status,participant_type,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,14.0,NaN,NaN,...,-79.8559,NaN,0::20,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,43.0,NaN,NaN,...,-118.3330,NaN,0::20,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,...,-82.1377,2.0,0::25||1::31||2::33||3::34||4::33,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,6.0,NaN,NaN,...,-104.8020,NaN,0::29||1::33||2::56||3::33,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,...,-79.9569,2.0,0::18||1::46||2::14||3::47,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,62.0,27.0


In [3]:
# Load the other CSVs
mental_health = pd.read_csv('EverytownNICSMentalHealth.csv')
gun_safety = pd.read_csv('GiffordsData.csv')
gun_ownership = pd.read_csv('Gun ownership.csv')
happiness = pd.read_csv('happiness.csv')
concealed_carry = pd.read_csv('ConcealedCarryPrices.csv')

In [4]:
# Happiness pre-processing: Add a new state column and update the city column
happiness['State'] = happiness['City'].str[-2:]
happiness['City'] = happiness['City'].str[:-4]

In [5]:
# Happiness pre-processing: converting two-letter abbrev. to state
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
# Add the full state name to the happiness table
happiness['State_full'] = happiness['State'].map(states)

# Create a second data table that counts how many happiness ratings we have for each state
happiness2 = happiness.groupby('State_full').count()['City'].to_frame()
happiness2 = happiness2.rename(index=str, columns={'City': 'Happiness Score'})

In [6]:
# WARNING: slow code block; takes a couple of minutes
# Extract the number of people in each incident
num_ages = gun_data['participant_age'].str.extractall('\d+::(\d+)').reset_index(level=1).groupby(level=0).max()['match']
num_genders = gun_data['participant_gender'].str.extractall('\d+::(\w+)').reset_index(level=1).groupby(level=0).max()['match']
num_names = gun_data['participant_name'].str.extractall('\d+::(\w+\s*\w*)').reset_index(level=1).groupby(level=0).max()['match']
num_relationships = gun_data['participant_relationship'].str.extractall('\d+::(\w+)').reset_index(level=1).groupby(level=0).max()['match']
num_statuses = gun_data['participant_status'].str.extractall('\d+::(\w+)').reset_index(level=1).groupby(level=0).max()['match']
num_types = gun_data['participant_type'].str.extractall('\d+::(\w+)').reset_index(level=1).groupby(level=0).max()['match']

# Get the number of people in each incident by taking the max the respective categories
num_merged = pd.concat([num_ages, num_genders, num_names, num_relationships, num_statuses, num_types], axis=1)
all_nums = num_merged.max(axis=1, skipna=True).astype(int)

# Make it into a df
all_nums = all_nums.to_frame()
all_nums.columns = ['n_people']

# Add 1 because Python is zero indexed
all_nums['n_people'] += 1

print(len(all_nums))

211490


In [7]:
# Extract the ages
raw_ages = gun_data['participant_age'].str.extractall('\d+::(?P<age>\d+)')
raw_ages['age'] = raw_ages['age'].astype(int)
raw_ages.reset_index(inplace=True)

# Count the ages in each group
ages = raw_ages.groupby(['level_0', pd.cut(raw_ages['age'], [0, 25, 45, 999])]).count()
ages.rename({'age': 'count'}, axis=1, inplace=True)
ages.fillna(0, axis=1, inplace=True)
ages['count'] = ages['count'].astype(int)

# Pivot the table and rename columns
ages = ages.reset_index().pivot(index='level_0', columns='age', values='count')
ages.index.rename('incident', inplace=True)
ages.columns = ages.columns.astype(str)
ages.rename({'(0, 25]': 'n_young', '(25, 45]': 'n_mid', '(45, 999]': 'n_old'}, axis=1, inplace=True)

print(len(ages))
ages.head()

145316


age,n_young,n_mid,n_old
incident,,,
0,1,0,0
1,1,0,0
2,1,4,0
3,0,3,1
4,2,0,2


In [8]:
# Extract the genders
raw_genders = gun_data['participant_gender'].str.extractall('\d+::(?P<gender>\w+)')
raw_genders.reset_index(inplace=True)

# Count the ages in each group
genders = raw_genders.groupby(['level_0', 'gender']).count()
genders.rename({'match': 'count'}, axis=1, inplace=True)
genders['count'] = genders['count'].astype(int)

# Pivot the table and rename columns
genders = genders.reset_index().pivot(index='level_0', columns='gender', values='count')
genders.fillna(0, axis=1, inplace=True)
genders.index.rename('incident', inplace=True)
genders.columns = genders.columns.astype(str)
genders.rename({'Female': 'n_female', 'Male': 'n_male'}, axis=1, inplace=True)
genders['n_female'] = genders['n_female'].astype(int)
genders['n_male'] = genders['n_male'].astype(int)

print(len(genders))
genders.head()

200201


gender,n_female,n_male
incident,,
0,1,3
1,0,1
2,0,5
3,1,3
4,2,2


In [9]:
# Join the data
joined_data = pd.concat([gun_data, all_nums, ages, genders], axis=1)

# Add the missing columns
joined_data['n_unknown_gender'] = joined_data['n_people'] - joined_data['n_female'] - joined_data['n_male']
joined_data['n_unknown_age'] = joined_data['n_people'] - joined_data['n_young'] - joined_data['n_mid'] - joined_data['n_old']

joined_data.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,congressional_district,gun_stolen,gun_type,...,state_house_district,state_senate_district,n_people,n_young,n_mid,n_old,n_female,n_male,n_unknown_gender,n_unknown_age
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,14.0,NaN,NaN,...,NaN,NaN,5.0,1.0,0.0,0.0,1.0,3.0,1.0,4.0
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,43.0,NaN,NaN,...,62.0,35.0,5.0,1.0,0.0,0.0,0.0,1.0,4.0,4.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,...,56.0,13.0,5.0,1.0,4.0,0.0,0.0,5.0,0.0,0.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,6.0,NaN,NaN,...,40.0,28.0,4.0,0.0,3.0,1.0,1.0,3.0,0.0,0.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,...,62.0,27.0,4.0,2.0,0.0,2.0,2.0,2.0,0.0,0.0


In [10]:
# Get incidents where there is no information about the people (aka missing gender or missing age data for everyone)
missing_data = joined_data[joined_data['n_unknown_gender'].isnull() | joined_data['n_unknown_age'].isnull()]
print(len(missing_data))

95548


In [ ]:
# CAMILLE'S OLD CODE; NOT USED ANYMORE

# # For each incident, generate column values for #female, #male, and various age groups
# gun_data['n_female'] = gun_data['n_killed'].copy()
# gun_data['n_male'] = gun_data['n_killed'].copy()
# gun_data['n_unknown_gender'] = gun_data['n_killed'].copy()
# gun_data['n_young'] = gun_data['n_killed'].copy()
# gun_data['n_mid'] = gun_data['n_killed'].copy()
# gun_data['n_old'] = gun_data['n_killed'].copy()
# gun_data['n_unknown_age'] = gun_data['n_killed'].copy()
# for i in range(len(gun_data)):
#     if i % 100 == 0:
#         print("processed:", i, "so far")
#     people = get_people(gun_data.iloc[i])
#     values = {'age': 1000, 'gender': "Unknown"}
#     people = people.fillna(value=values)
#     gender_people = people.groupby('gender').count()['age'].to_frame().rename(index=str, columns={"age": "Count"})
#     age_people = people.groupby('age').count()['gender'].to_frame().rename(index=str, columns={"gender": "Count"})
#     age_people['Age'] = age_people.index
#     try: 
#         gun_data['n_female'][i] = gender_people.loc["Female"][0]
#     except:
#         gun_data['n_female'][i] = 0
#     try: 
#         gun_data['n_male'][i] = gender_people.loc["Male"][0]
#     except:
#         gun_data['n_male'][i] = 0
#     try: 
#         gun_data['n_unknown_gender'][i] = gender_people.loc["Unknown"][0]
#     except:
#         gun_data['n_unknown_gender'][i] = 0
#     count_young = 0
#     count_mid = 0
#     count_old = 0
#     count_unknown = 0
#     for j in range(len(age_people)):
#         age = float(age_people.iloc[j][1])
#         count = age_people.iloc[j][0]
        
#         if age < 25:
#             count_young += count
#         elif age < 45:
#             count_mid += count
#         elif age < 999:
#             count_old += count
#         else:
#             count_unknown += count
#     gun_data['n_young'][i] = count_young
#     gun_data['n_mid'][i] = count_mid
#     gun_data['n_old'][i] = count_old
#     gun_data['n_unknown_age'][i] = count_unknown

In [11]:
# Merge Everything
merged1 = joined_data.merge(mental_health, left_on='state', right_on='State')
merged2 = merged1.merge(gun_safety, left_on='state', right_on='State')
merged3 = merged2.merge(gun_ownership, left_on='state', right_on='State')
merged4 = merged3.merge(concealed_carry, left_on='state', right_on='State')
merged5 = merged4.merge(happiness2, left_on='state', right_on='State_full')
merged5.columns

Index(['incident_id', 'date', 'state', 'city_or_county', 'address', 'n_killed',
       'n_injured', 'congressional_district', 'gun_stolen', 'gun_type',
       'incident_characteristics', 'latitude', 'longitude', 'n_guns_involved',
       'participant_age', 'participant_gender', 'participant_name',
       'participant_relationship', 'participant_status', 'participant_type',
       'state_house_district', 'state_senate_district', 'n_people', 'n_young',
       'n_mid', 'n_old', 'n_female', 'n_male', 'n_unknown_gender',
       'n_unknown_age', 'State_x', 'Mental_Health_Records_Submitted_2008',
       'Mental_Health_Records_Submitted_2017', 'Gun_Sale_Denials_2008',
       'Gun_Sale_Denials_2017', 'State_y', 'Giffords_Gun_Safety_Grade',
       'Gun_Death_Rate_2018 (per 100k people)', 'Gun_Death_Rank', 'Rank',
       'State_x', '# of guns per capita', '# of guns registered', 'State_y',
       'Permit Type', 'Handgun_Carry_Permit_Fee', 'Years_Valid', '5_Year_Cost',
       'Happiness Score'],
 

In [27]:
final_data = merged5[(merged5['n_killed'] != 0) & (merged5['n_injured'] != 0)]

# Changing format of date column
final_data.loc[:,'year'] = pd.DatetimeIndex(final_data.loc[:,'date']).year
final_data.loc[:,'month'] = pd.DatetimeIndex(final_data.loc[:,'date']).month
final_data.loc[:,'day'] = pd.DatetimeIndex(final_data.loc[:,'date']).day
cols = final_data.columns.tolist()
cols = cols[:1] + cols[-3:] + cols[2:-3]
final_data = final_data[cols]
final_data.head()

final_data.to_csv('final_data.csv')

In [28]:
final_data.head()

,incident_id,year,month,day,state,city_or_county,address,n_killed,n_injured,congressional_district,...,State_x,# of guns per capita,# of guns registered,State_y,State_y,Permit Type,Handgun_Carry_Permit_Fee,Years_Valid,5_Year_Cost,Happiness Score
1,484268,2013,4,9,Pennsylvania,Philadelphia,Somerset Street and Lee Street,1,3,1.0,...,Pennsylvania,18.45,"236,377",Pennsylvania,Pennsylvania,License to carry firearms,20,5,20,2
2,485885,2013,4,28,Pennsylvania,Chester,Rose and Upland Street,1,3,1.0,...,Pennsylvania,18.45,"236,377",Pennsylvania,Pennsylvania,License to carry firearms,20,5,20,2
8,491609,2013,8,6,Pennsylvania,Saylorsburg,1000 Ross Municipal Dr,3,4,17.0,...,Pennsylvania,18.45,"236,377",Pennsylvania,Pennsylvania,License to carry firearms,20,5,20,2
11,480961,2013,10,6,Pennsylvania,Philadelphia,54th St. and Florence Ave,1,5,2.0,...,Pennsylvania,18.45,"236,377",Pennsylvania,Pennsylvania,License to carry firearms,20,5,20,2
14,481268,2013,10,21,Pennsylvania,Pittsburgh (Homewood),7200 block of Kelly Street,2,5,14.0,...,Pennsylvania,18.45,"236,377",Pennsylvania,Pennsylvania,License to carry firearms,20,5,20,2
